In [1]:
# Used a lot of code out of the examples of ipycanvas

from ipycanvas import Canvas, hold_canvas
from ipywidgets import HBox, VBox, IntSlider, Image, Text, Label, RadioButtons, Button, Output, Tab, FloatSlider, FileUpload, Dropdown

from mmgen.apis import init_model, sample_uncoditional_model  # isort:skip  # noqa
from mmgen.models.architectures import positional_encoding

import numpy as np
import matplotlib.pyplot as plt
from torchvision import utils

import torch
from scipy import ndimage
import warnings
import os

from IPython.display import display, clear_output

from PIL import Image

def displace(inputs, level):
    # return torch.from_numpy(ndimage.rotate(inputs, x, axes=axes,reshape=False))
    if level:
        return inputs + torch.normal(0, level/1000,size=inputs.shape)
    else:
        return inputs


def warp(inputs, base_x, base_y, magnitude=1):
    new_img = torch.zeros_like(inputs)
    new_img[0] = (inputs[0] - base_x)
    new_img[1] = (inputs[1] - base_y)

    distances = torch.sqrt(new_img[0]**2 + new_img[1]**2) 

    new_img[0] = base_x + new_img[0]/distances * (distances ** magnitude) # /new_img[0]**2
    new_img[1] = base_y + new_img[1]/distances * (distances ** magnitude) # /new_img[1]**2
    return new_img

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
dislevel = 0
x,y = 0, 0
size = 1024
no_points = 10
is_style=False

In [3]:
model_list = sorted(list(os.listdir("checkpoints")))
print(model_list)
def load_model(run_name):
    config = f"configs/scaleparty/{run_name}.py"
    ckpt = f"checkpoints/{run_name}/ckpt/latest.pth"
    device='cpu' if not torch.has_cuda else "cuda"
    use_noise = True


    model = init_model(
        config, checkpoint=ckpt, device=device)

    get_pos = model.generator.head_pos_enc.make_grid2d
    return model, get_pos, config

model, get_pos, config = load_model(model_list[0])

['FFHQ_ScaleParty', 'FFHQ_ScaleParty.zip']
0.2
Use load_from_local loader
The model and loaded state dict do not match exactly

size mismatch for generator.injected_noise_0: copying a param with shape torch.Size([1, 1, 4, 4]) from checkpoint, the shape in current model is torch.Size([1, 1, 8, 8]).
size mismatch for generator.injected_noise_1: copying a param with shape torch.Size([1, 1, 10, 10]) from checkpoint, the shape in current model is torch.Size([1, 1, 14, 14]).
size mismatch for generator.injected_noise_2: copying a param with shape torch.Size([1, 1, 8, 8]) from checkpoint, the shape in current model is torch.Size([1, 1, 12, 12]).
size mismatch for generator.injected_noise_3: copying a param with shape torch.Size([1, 1, 18, 18]) from checkpoint, the shape in current model is torch.Size([1, 1, 22, 22]).
size mismatch for generator.injected_noise_4: copying a param with shape torch.Size([1, 1, 16, 16]) from checkpoint, the shape in current model is torch.Size([1, 1, 20, 20]).
siz

2022-06-06 15:55:25,584 - mmgen - INFO - Switch to evaluation style mode: single
2022-06-06 15:55:25,585 - mmgen - INFO - Switch to evaluation style mode: single


In [4]:
canvas = Canvas(width=512, height=512)
canvas.fill_style = '#584f4e'
canvas.fill_rect(128, 128, 256, 256)

warp_x = 256
warp_y = 256


objects_to_draw = []
positions = None 
class Square_obj():
    def __init__(self, x,y, width=5, height=5, append_self = True):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.selected = False
        if append_self:
            objects_to_draw.append(self)
        
        
        
    def set_x_y(self,x_in,y_in)  :
        self.x = x_in
        self.y = y_in
    
    def draw(self, padding=False):
        
        # canvas.fill_style = '#38a8a4'
        # canvas.fill_rect(self.x - (self.width*0.5), self.y - (self.height) , self.width, self.height)
        if padding:
            canvas.fill_style = '#dd2e06'
        else:
            canvas.fill_style = '#2edd0c'
        canvas.fill_rect(self.x - (self.width*0.5), self.y - (self.height*0.5) , self.width, self.height)
        # canvas.fill_circle(x,y,radius=self.width)

        
        
    def is_selected(self,x_in, y_in):
        x_coord = self.x - (self.width*0.5)
        y_coord = self.y - (self.height*0.5)

        if x_in > x_coord and x_in < (x_coord+ self.width) and  y_in > y_coord  and y_in < (y_coord  + self.height):
            
            self.set_selected(True)
            return True
        else:
            self.set_selected(False)
            return False
    
    def set_selected(self,state):
        self.selected = state

class Warp_obj(Square_obj):
    def __init__(self, x, y, width=5, height=5):
        super().__init__(x, y, width=width, height=height, append_self=False)

    def draw(self):
        canvas.fill_style = "#deaaaa"
        canvas.fill_rect(self.x - (self.width*0.5), self.y - (self.height*0.5) , self.width, self.height)

def canvas_restart():
    canvas.clear()
    canvas.fill_style = '#584f4e'
    canvas.fill_rect(128, 128, 256, 256)

def canvas_init(x,y,size,dislevel,number, ratio=1.0, magnitude=1.0):
    global positions
    global warpoint
    canvas_restart()

    size_x = size
    size_y = size*ratio

    positions = get_pos(n_height=number-6, 
                        n_width=number-6,
                        start_w=(x-size_x/2),
                        start_h=(y-size_y/2),
                        width=size_x,
                        height=size_y
                        )[0]
    


    positions = displace(positions, dislevel)
     
    positions = warp(positions, (warpoint.x - 256)/128, (warpoint.y-256)/128, magnitude=magnitude)

    positions = torch.nan_to_num(positions)
    with hold_canvas(canvas):
        cnt = 0
        for pos in positions.view(2,number**2).transpose(0,1):
            x,y = pos.numpy()
            s=Square_obj(x*128 + 256 ,y*128 + 256)
            s.set_selected(False)
            s.draw(padding= (cnt % number) < 3 or (cnt % number > number - 4) or (cnt // number < 3) or (cnt // number > number - 4))
            cnt += 1
    warpoint.draw()

def handle_mouse_up(x, y):
    global warpoint
    warpoint.set_selected(False)


def handle_mouse_down(x, y):
    global warpoint, warp_x, warp_y
    
    warp_y = y
    warp_x = x
    

    warpoint.set_x_y(x,y)

    on_slider_change()


def handle_mouse_move(x, y):    
    global warpoint
    warpoint.set_x_y(x,y)




warpoint = Warp_obj(warp_x, warp_y)
warpoint.set_selected = True 
warpoint.draw()

# canvas.on_mouse_move(handle_mouse_move)
# canvas.on_mouse_up(handle_mouse_up)

canvas_init(0.5,0.5,1.,0,10)
canvas.on_mouse_down(handle_mouse_down)


In [5]:
# from interactive_utils import generate

# latent = get_latent(g_ema, n_samples=1)
# mean_latent = model.get_mean_latent(args.truncation_mean, device = device)
if "mspie" in config:
    latent = model(None, num_batches=1, chosen_scale=0, return_latents=True)["noise_batch"]
else:
    latent = model.generator.get_noise(1)

is_style=False
def generate():

    extra = dict(truncation=0.7)
    if "mspie" in config:
        results = model.generator_ema(
            latent, num_batches=1, chosen_scale=slider_number.value-10,
            truncation=slider_trunc.value)
    else:
        results = model.generator_ema(
            latent, num_batches=1, chosen_scale=slider_number.value-10, input_is_latent = is_style,
            positional_enc = positions.unsqueeze(0),truncation=slider_trunc.value)
 

    results = (results[:, [2, 1, 0]] + 1.) / 2.

    return results[0].detach().cpu().numpy().transpose((1,2,0))


out = Output()
t = Tab()
t.children = [out]

plt.ioff()
ax=plt.gca()
ax.figure.set_size_inches((8,8))
ax.set_axis_off()

plt.ion()
with out:
    warnings. simplefilter("ignore")
    ax.imshow(generate())

    


In [6]:
text_size = Label(
    value='Zoom',
    placeholder='Image size',
    description='String: pick the size of the image',
    disabled=False
)
slider_size = FloatSlider(min=0.2, max =2.5, value= 1.0,step=0.01)

text_ratio = Label(
    value='Aspect ratio',
    placeholder='Image ratio',
    description='String: pick the ratio of the image',
    disabled=False
)
slider_ratio = FloatSlider(min=0.2, max =5, value= 1, step = 0.1)

text_attract = Label(
    value='Attract magnitude',
    placeholder='Image attract',
    description='String: pick the attract of the image',
    disabled=False
)
slider_attract = FloatSlider(min=0.2, max =5, value= 1, step = 0.1)

text_x = Label(
    value='Position x',
    placeholder='Image x',
    description='String: pick the x of the image',
    disabled=False
)

slider_x = FloatSlider(min=-0.5, max =1.5, value = 0.5, step=0.01)

text_y = Label(
    value='Position y',
    placeholder='Image y',
    description='String: pick the y of the image',
    disabled=False
)
slider_y = FloatSlider(min=-0.5, max =1.5, value = 0.5, step=0.01)

text_rotation = Label(
    value='Noisy displacement',
    placeholder='Image rotation',
    description='String: pick the rotation of the image',
    disabled=False
)
slider_rotation = IntSlider(min=0, max =360)

text_number = Label(
    value='Number or points (NxN)',
    placeholder='Image number',
    description='String: pick the number of the image',
    disabled=False
)
slider_number = IntSlider(min=7, max =24, value= 12)

click_options = RadioButtons(
    options=['Move', 'Attract', 'Repel'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Click behavior:',
    disabled=False
)

gen_button = Button(
    description='Generate',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='' # (FontAwesome names without the `fa-` prefix)
)
latent_button = Button(
    description='Sample Latent',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='' # (FontAwesome names without the `fa-` prefix)
)

fuploader = FileUpload(
    accept='.pkl',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

model_dropdown = Dropdown(
    options = model_list,
    value=model_list[0],
    disabled=False
    )

text_trunc = Label(
    value='Truncation',
    description='level of truncation for latent code',
    disabled=False
)

slider_trunc = FloatSlider(min=0.0, max =1.0, value = 0.8, step=0.05)


def on_slider_change(change=None):
    canvas_init(slider_x.value, slider_y.value, slider_size.value, slider_rotation.value, slider_number.value, slider_ratio.value, slider_attract.value)

@out.capture()
def on_generate_click(button):
    global tmp_img
    ax.clear()
    tmp_img = generate()
    ax.imshow(tmp_img)
    # plt.clear()
    with out:
        # im.set_data(generate())
        clear_output(wait=True)
        # fig.canvas.draw_idle()
        # ax.show()
        # plt.show()
        display(ax.figure)
    # fig.canvas.flush_events()
    



def on_latent_click(button):
    global latent
    # latent = get_latent(g_ema, n_samples=1)
    # latent = model.get_mean_latent()
    if "mspie" in config:
        latent = model(None, num_batches=1, chosen_scale=0, return_latents=True)["noise_batch"]
    else:
        latent = model.generator.get_noise(1)

def on_upload_change(change):
    global latent
    latent = torch.load(fuploader.value)

def on_dropdown_change(change):
    global model, get_pos, config
    model, get_pos, config = load_model(model_dropdown.value)


slider_x.observe(on_slider_change, 'value')
slider_y.observe(on_slider_change, 'value')
slider_size.observe(on_slider_change, 'value')
slider_ratio.observe(on_slider_change, 'value')
slider_attract.observe(on_slider_change, 'value')
slider_rotation.observe(on_slider_change, 'value')
slider_number.observe(on_slider_change, 'value')

fuploader.observe(on_upload_change, names='_counter')

model_dropdown.observe(on_dropdown_change, 'value')


latent_button.on_click(on_latent_click)
gen_button.on_click(on_generate_click)

buttons = HBox((latent_button, gen_button))
trunc_box = HBox((text_trunc, slider_trunc))
controls = VBox((text_x, slider_x, text_y, slider_y, text_size, slider_size, text_ratio, slider_ratio, text_rotation, slider_rotation, text_number, slider_number, text_attract, slider_attract, buttons, model_dropdown, trunc_box))
# controls = VBox((text_x, slider_x, text_y, slider_y, text_size, slider_size, text_ratio, slider_ratio, text_number, slider_number, text_attract, slider_attract, buttons, model_dropdown, trunc_box))



### Experiment with different configuration of positional encodings

The networks was trained with different crops of images at 256 and 384 pixel resolution. The crops were between 60% and 110% of the scale of a full-portrait. 
Which mean that the network has seen during training full-frame resolution of ~232px to 640px. However, it was never trained to generate a full-frame images larger than 384. Going beyond the 232-640 scale range can produce erroneous results. Moreover, the _aspect ratio_ and _attract magnitude_ will also produce configurations unseen during training. Use noisy displacement for some trippy results :D. 

In [7]:

is_style=False
display(HBox([canvas, controls, out]))


### Save the output

In [8]:
canvas.sync_image_data = True
imgname = "scaleparty_sample"
from PIL import Image

canvas.to_file(f"visuals/{imgname}_posenc.png")


im = Image.fromarray((255*np.clip(tmp_img,0,1)).astype(np.uint8))

im.save(f"visuals/{imgname}_img.png")


RuntimeError: No image data to save, please be sure that ``sync_image_data`` is set to True